This notebook imports the GPR model published in the article titled "Machine Learning for Predictive Understanding of Spall Strength of Metals and Alloys" and offers predictions for the spall strength of new candidates. For more detailed information, please refer to
[hyperlink](paper).

In [2]:
#@title Predict the Spall Strength for a Metal or Alloy
#@markdown ***Please first input your parameters, and then press on the left button to run.***

# Import packages
import os
import pickle
import pandas as pd
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

# Import data
if os.path.exists("/usr/local/spall_strength_prediction_models"):
  pass
else:
  %cd /usr/local
  !git clone -q https://github.com/Ramprasad-Group/spall_strength_prediction.git



# @markdown 1. Ultimate Tensile Strength (UTS) (MPa):
UTS = 159 # @param {type:"number"}
# @markdown 2. Tensile Yield Strength (MPa):
yield_strength = 70 # @param {type:"number"}
# @markdown 3. Bulk Modulus (GPa):
bulk_modulus = 101 # @param {type:"number"}
# @markdown 4. Young's Modulus (GPa):
youngs_modulus = 75 # @param {type:"number"}
# @markdown 5. Density (g/cc):
density = 10.50 # @param {type:"number"}
# @markdown 6. Vickers Hardness (HV):
hardness = 25 # @param {type:"number"}
# @markdown 7. Fracture Toughness (MPa√m):
fracture_toughness = 73 # @param {type:"number"}

# Create a pandas DataFrame
column_names = ['UTS', 'yield_strength', 'bulk_modulus', 'youngs_modulus', 'density',
       'hardness', 'fracture_toughness']
input_data_list = [UTS,yield_strength, bulk_modulus, youngs_modulus, density,
                   hardness, fracture_toughness]
input_data = pd.DataFrame(columns=column_names)
input_data.loc[len(input_data)] = input_data_list

### Descriptor scaling ###
train = pd.read_csv("/usr/local/spall_strength_prediction/datasets/train.csv")
trainX = train.to_numpy()[:,3:]
trainY = train.to_numpy()[:,2]
testX = input_data.to_numpy()
scalerM = MinMaxScaler()
scalerM.fit(trainX)
trainX_M = scalerM.transform(trainX)
testX_M = scalerM.transform(testX)

# Build the GPR model
gpr_final = GaussianProcessRegressor(alpha=2,
                                     kernel=3.5**2 * RBF(length_scale=.355)  ,
                                     n_restarts_optimizer = 10,
                                     normalize_y=True,
                                     optimizer=None,
                                     random_state=1
                                    )

GPR = gpr_final.fit(trainX_M, trainY);


#GPR
test_pred_GPR, test_pred_GPR_std = GPR.predict(testX_M, return_std=True)


print('Predicted Spall Strength using Gaussian Process Regression (GPR) model is: %.3f ± %.3f GPa' %(test_pred_GPR[0], test_pred_GPR_std[0]))



/usr/local
fatal: could not read Username for 'https://github.com': No such device or address


FileNotFoundError: [Errno 2] No such file or directory: '/Users/keara/Dropbox (GaTech)/spall/for_Github/datasets/train.csv'

# References and further reading

- Keara Frawley, Harikrishna Sahu, Naresh Thadhani, Rampi Ramprasad, *Machine Learning for Predictive Understanding of Spall Strength of Metals and Alloys*, citation info, DOI: [hyperlink](paper)